In [1]:
%load_ext autoreload
%autoreload 2
from entities.car import Car
from entities.stoplight import Stoplight

import pygame
import random

pygame.init()

WINDOW_WIDTH = 1000
WINDOW_HEIGHT = 1000
window = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Intersection Simulation")

BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0, 100)
GREEN = (0, 255, 0, 100)
YELLOW = (255, 255, 0, 100)
GRAY = (60, 60, 60)

ambient_1 = pygame.image.load('assets/ambient_1.png')
ambient_2 = pygame.image.load('assets/ambient_2.png')
ambient_3 = pygame.image.load('assets/ambient_3.png')
ambient_4 = pygame.image.load('assets/ambient_4.png')

try:
    pygame.mixer.init()
    # Set audio effect in loop
    pygame.mixer.music.load('assets/Street Sound Effect.mp3')
    pygame.mixer.music.play(-1)
except pygame.error as e:
    print(f"Warning: {e}")

# Load image, resize it to be 370x370 and put it in the right corner
ambient_1 = pygame.transform.scale(ambient_1, (WINDOW_WIDTH//2 - 30, WINDOW_HEIGHT//2 - 30))
ambient_2 = pygame.transform.scale(ambient_2, (WINDOW_WIDTH//2 - 30, WINDOW_HEIGHT//2 - 30))
ambient_3 = pygame.transform.scale(ambient_3, (WINDOW_WIDTH//2 - 30, WINDOW_HEIGHT//2 - 30))
ambient_4 = pygame.transform.scale(ambient_4, (WINDOW_WIDTH//2 - 30, WINDOW_HEIGHT//2 - 30))
window.blit(ambient_1, (0, 0))
window.blit(ambient_2, (WINDOW_WIDTH//2 + 30, 0))
window.blit(ambient_3, (WINDOW_WIDTH//2 + 30, WINDOW_HEIGHT//2 + 30))
window.blit(ambient_4, (0, WINDOW_HEIGHT//2 + 30))

   

run = True
clock = pygame.time.Clock()

cars = []
prev_time = 0
stoplight = Stoplight(random.choice([GREEN, RED]))

while run:

    clock.tick(30)
    stoplight.update_stoplight()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False

    time = pygame.time.get_ticks() // 1000

    if time != prev_time:
        if time % 2 == 0:
            direction = random.choice(['up', 'down', 'left', 'right'])
            if direction == 'up':
                cars.append(Car(WINDOW_WIDTH//2 + 5, WINDOW_HEIGHT, direction))
            elif direction == 'down':
                cars.append(Car(WINDOW_WIDTH//2 - 20 - 4, 0, direction))
            elif direction == 'left':
                cars.append(Car(WINDOW_WIDTH, WINDOW_HEIGHT//2 - 20 - 4, direction))
            elif direction == 'right':
                cars.append(Car(0, WINDOW_HEIGHT//2 + 5, direction))
        prev_time = time

    if stoplight.time_green >= 300:
        stoplight.switch_yellow()
    
    for car in cars:
        if not car.stopped:
            if ((car.direction == 'up' and car.y == WINDOW_HEIGHT//2 + 50 and (stoplight.color_NS == RED or stoplight.color_NS == YELLOW)) or
                (car.direction == 'down' and car.y + car.length == WINDOW_HEIGHT//2 - 50 and (stoplight.color_NS == RED or stoplight.color_NS == YELLOW)) or
                (car.direction == 'left' and car.x == WINDOW_WIDTH//2 + 50 and (stoplight.color_EW == RED or stoplight.color_EW == YELLOW)) or
                (car.direction == 'right' and car.x + car.length == WINDOW_WIDTH//2 - 50 and (stoplight.color_EW == RED or stoplight.color_EW == YELLOW))) or not car.can_move(cars):
                car.stop()
            else:
                if ((car.direction == 'up' and car.y <= WINDOW_HEIGHT//2 + car.speed and car.y >= WINDOW_HEIGHT//2 - car.speed) or
                    (car.direction == 'down' and car.y + car.length >= WINDOW_HEIGHT//2 - car.speed and car.y + car.length <= WINDOW_HEIGHT//2 + car.speed) or
                    (car.direction == 'left' and car.x <= WINDOW_WIDTH//2 + car.speed and car.x >= WINDOW_WIDTH//2 - car.speed) or
                    (car.direction == 'right' and car.x + car.length >= WINDOW_WIDTH//2 - car.speed and car.x + car.length <= WINDOW_WIDTH//2 + car.speed)):
                    car.turn_or_straight()
                car.move()
        elif car.stopped and ((car.direction in ['up', 'down'] and stoplight.color_NS == GREEN) or (car.direction in ['left', 'right'] and stoplight.color_EW == GREEN)):
            car.stopped = False
            car.move()
        if car.stopped:
            car.waiting_time += 1
        else:
            car.waiting_time = 0

    cars = [car for car in cars if 0 <= car.x <= WINDOW_WIDTH and 0 <= car.y <= WINDOW_HEIGHT]

    # Draw intersection
    ## Draw road
    pygame.draw.line(window, GRAY, (0, WINDOW_HEIGHT//2), (WINDOW_WIDTH, WINDOW_HEIGHT//2), 60)
    pygame.draw.line(window, GRAY, (WINDOW_WIDTH//2, 0), (WINDOW_WIDTH//2, WINDOW_HEIGHT), 60)
    ## Draw lanes
    pygame.draw.line(window, WHITE, (0, WINDOW_HEIGHT//2 - 28), (WINDOW_WIDTH, WINDOW_HEIGHT//2 - 28), 1)
    pygame.draw.line(window, WHITE, (0, WINDOW_HEIGHT//2 + 28), (WINDOW_WIDTH, WINDOW_HEIGHT//2 + 28), 1)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 28, 0), (WINDOW_WIDTH//2 - 28, WINDOW_HEIGHT), 1)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 28, 0), (WINDOW_WIDTH//2 + 28, WINDOW_HEIGHT), 1)
    pygame.draw.line(window, WHITE, (0, WINDOW_HEIGHT//2), (WINDOW_WIDTH, WINDOW_HEIGHT//2), 4)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2, 0), (WINDOW_WIDTH//2, WINDOW_HEIGHT), 4)
    ## Draw crosswalks
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 23, WINDOW_HEIGHT//2 - 200), (WINDOW_WIDTH//2 - 23, WINDOW_HEIGHT//2 - 180), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 17, WINDOW_HEIGHT//2 -200), (WINDOW_WIDTH//2 - 17, WINDOW_HEIGHT//2 - 180), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 12, WINDOW_HEIGHT//2 - 200), (WINDOW_WIDTH//2 - 12, WINDOW_HEIGHT//2 - 180), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 6, WINDOW_HEIGHT//2 - 200), (WINDOW_WIDTH//2 - 6, WINDOW_HEIGHT//2 - 180), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 6, WINDOW_HEIGHT//2 - 200), (WINDOW_WIDTH//2 + 6, WINDOW_HEIGHT//2 - 180), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 12, WINDOW_HEIGHT//2 - 200), (WINDOW_WIDTH//2 + 12, WINDOW_HEIGHT//2 - 180), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 17, WINDOW_HEIGHT//2 - 200), (WINDOW_WIDTH//2 + 17, WINDOW_HEIGHT//2 - 180), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 23, WINDOW_HEIGHT//2 - 200), (WINDOW_WIDTH//2 + 23, WINDOW_HEIGHT//2 - 180), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 200, WINDOW_HEIGHT//2 - 23), (WINDOW_WIDTH//2 - 180, WINDOW_HEIGHT//2 - 23), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 200, WINDOW_HEIGHT//2 - 17), (WINDOW_WIDTH//2 - 180, WINDOW_HEIGHT//2 - 17), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 200, WINDOW_HEIGHT//2 - 12), (WINDOW_WIDTH//2 - 180, WINDOW_HEIGHT//2 - 12), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 200, WINDOW_HEIGHT//2 - 6), (WINDOW_WIDTH//2 - 180, WINDOW_HEIGHT//2 - 6), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 200, WINDOW_HEIGHT//2 + 6), (WINDOW_WIDTH//2 - 180, WINDOW_HEIGHT//2 + 6), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 200, WINDOW_HEIGHT//2 + 12), (WINDOW_WIDTH//2 - 180, WINDOW_HEIGHT//2 + 12), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 200, WINDOW_HEIGHT//2 + 17), (WINDOW_WIDTH//2 - 180, WINDOW_HEIGHT//2 + 17), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 200, WINDOW_HEIGHT//2 + 23), (WINDOW_WIDTH//2 - 180, WINDOW_HEIGHT//2 + 23), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 180, WINDOW_HEIGHT//2 - 23), (WINDOW_WIDTH//2 + 200, WINDOW_HEIGHT//2 - 23), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 180, WINDOW_HEIGHT//2 - 17), (WINDOW_WIDTH//2 + 200, WINDOW_HEIGHT//2 - 17), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 180, WINDOW_HEIGHT//2 - 12), (WINDOW_WIDTH//2 + 200, WINDOW_HEIGHT//2 - 12), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 180, WINDOW_HEIGHT//2 - 6), (WINDOW_WIDTH//2 + 200, WINDOW_HEIGHT//2 - 6), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 180, WINDOW_HEIGHT//2 + 6), (WINDOW_WIDTH//2 + 200, WINDOW_HEIGHT//2 + 6), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 180, WINDOW_HEIGHT//2 + 12), (WINDOW_WIDTH//2 + 200, WINDOW_HEIGHT//2 + 12), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 180, WINDOW_HEIGHT//2 + 17), (WINDOW_WIDTH//2 + 200, WINDOW_HEIGHT//2 + 17), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 180, WINDOW_HEIGHT//2 + 23), (WINDOW_WIDTH//2 + 200, WINDOW_HEIGHT//2 + 23), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 23, WINDOW_HEIGHT//2 + 200), (WINDOW_WIDTH//2 - 23, WINDOW_HEIGHT//2 + 220), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 17, WINDOW_HEIGHT//2 + 200), (WINDOW_WIDTH//2 - 17, WINDOW_HEIGHT//2 + 220), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 12, WINDOW_HEIGHT//2 + 200), (WINDOW_WIDTH//2 - 12, WINDOW_HEIGHT//2 + 220), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 - 6, WINDOW_HEIGHT//2 + 200), (WINDOW_WIDTH//2 - 6, WINDOW_HEIGHT//2 + 220), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 6, WINDOW_HEIGHT//2 + 200), (WINDOW_WIDTH//2 + 6, WINDOW_HEIGHT//2 + 220), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 12, WINDOW_HEIGHT//2 + 200), (WINDOW_WIDTH//2 + 12, WINDOW_HEIGHT//2 + 220), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 17, WINDOW_HEIGHT//2 + 200), (WINDOW_WIDTH//2 + 17, WINDOW_HEIGHT//2 + 220), 2)
    pygame.draw.line(window, WHITE, (WINDOW_WIDTH//2 + 23, WINDOW_HEIGHT//2 + 200), (WINDOW_WIDTH//2 + 23, WINDOW_HEIGHT//2 + 220), 2)
    ## Cover intersection
    pygame.draw.rect(window, GRAY, (WINDOW_WIDTH//2 - 29, WINDOW_HEIGHT//2 - 29, 60, 60))
    ## Draw stoplights as stop lines
    pygame.draw.line(window, stoplight.color_NS, (WINDOW_WIDTH//2 - 27, WINDOW_HEIGHT//2 - 32), (WINDOW_WIDTH//2 - 2, WINDOW_HEIGHT//2 - 32), 5)
    pygame.draw.line(window, stoplight.color_NS, (WINDOW_WIDTH//2 + 3, WINDOW_HEIGHT//2 + 33), (WINDOW_WIDTH//2 + 27, WINDOW_HEIGHT//2 + 33), 5)
    pygame.draw.line(window, stoplight.color_EW, (WINDOW_WIDTH//2 - 32, WINDOW_HEIGHT//2 + 3), (WINDOW_WIDTH//2 - 32, WINDOW_HEIGHT//2 + 27), 5)
    pygame.draw.line(window, stoplight.color_EW, (WINDOW_WIDTH//2 + 33, WINDOW_HEIGHT//2 - 27), (WINDOW_WIDTH//2 + 33, WINDOW_HEIGHT//2 - 2), 5)
    
    # Draw cars
    for car in cars:
        car.draw(window)
    
    pygame.display.update()

pygame.quit()



pygame 2.6.0 (SDL 2.28.4, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


2024-07-22 13:47:13.076 Python[8797:9644072] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


: 